In [1]:
import pandas as pd
import numpy as np
import ssl
import urllib.request, urllib.parse, urllib.error
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import glob
import os.path
import re
from pandas.errors import EmptyDataError



In [2]:
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")


In [3]:
#rochelle.columns
namelist= []
for name in rochelle['Scientific Name']:
    namelist.append(name)

302


In [4]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [5]:
#accessing search page of the herbarium
atlas_url = "https://bellatlas.umn.edu/collections/harvestparams.php"
#atlas_html = urllib.request.urlopen(atlas_url, context=ctx).read()



In [6]:
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?
rounds = 0
emptyresults = 0
masterdataframe = pd.DataFrame()
driver.maximize_window()
scraped_dataframe = pd.DataFrame()#(index=[0])
scraped_dataframe.reset_index(drop=True)
for speciesname in namelist[:]:
    driver.get(atlas_url) # Goes to atlas
    namebox = driver.find_element(by = By.XPATH, value = '//*[@id="taxa"]')
    searchbutton = driver.find_element(by = By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/form/div[2]/div[1]/button') #selects the advanced search button based off the pulled tag from right clicking the inspected element
    time.sleep(.5)
    namebox.clear()
    namebox.send_keys(speciesname) #'Cabomba caroliniana') #Sends in a name from Rochelle's list
    searchbutton.click()

    resultcount = driver.find_element(by=By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[4]/div[3]').text
    resultcountnumberlist= list(map(int, list(filter(lambda x: x!= '', re.findall(r'\d*', re.search(r'\d* of \d*', resultcount).group()))))) #first number in list is the total results displayed on the page, the second number is the total results
#if there are not results
    if resultcountnumberlist[1] == 0:
        emptyresults+=1
        continue
#continue to next search
    
#if there are results
    windowbefore = driver.window_handles[0]
    downloadbutton = driver.find_element(by=By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[1]/form/button')
    downloadbutton.click()
    windowafter = driver.window_handles[1]
    driver.switch_to.window(windowafter)
    time.sleep(2)
    driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/form/fieldset/table/tbody/tr[4]/td[2]/div/input[2]').click() #utf8 encoding
    driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/form/fieldset/table/tbody/tr[5]/td[2]/div/input').click() #uncompresses download

    downloadbutton=driver.find_element(by=By.NAME, value= "submitaction") #OVERWRITES THE PREVIOUS DOWNLOAD BUTTON
    downloadbutton.click()

    folder_path = r'C:\Users\justi\Downloads'
    file_type = r'\*CSV'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)

    try:
        downloadeddata = pd.read_csv(max_file, encoding='utf-8')
    except:
        try:
            print('Empty Data Error Occured on file ' + max_file)
            downloadeddatafile = open(max_file, 'r')
            downloadeddata = pd.read_csv(downloadeddatafile, encoding='utf-8')
            downloadeddatafile.close()
            continue
        except:
            print("Skipping to next interation")
            continue
        
    
    if masterdataframe.empty:
        masterdataframe = pd.DataFrame(data=downloadeddata)
    else:
        masterdataframe = masterdataframe.append(pd.DataFrame(data = downloadeddata))
    driver.switch_to.window(windowbefore)
    rounds +=1
    os.remove(max_file)
    

Empty Data Error Occured on file C:\Users\justi\Downloads\1681703867-occur.csv
Skipping to next interation
Empty Data Error Occured on file C:\Users\justi\Downloads\1681704575-occur.csv
Skipping to next interation


In [108]:
file= masterdataframe.to_csv(r'C:\Users\justi\Downloads\minnatlas-out.csv', encoding='utf-8')

In [118]:
clienttemplate = pd.read_excel(r"C:\Users\justi\Downloads\GLANSISSpecimenBulkUploadTemplateTEST.xlsx").drop('Unnamed: 0', axis=1)
clienttemplate[['Genus*', 'Species*', 'Latitude*', 'Longitude*', 'Year*', 'Month', 'Day', 'Collectors', 'Reference ID*', 'Reference 3', 'Number collected/observed', 'Locality*'  ]] = masterdataframe[['genus', 'specificEpithet', 'decimalLatitude', 'decimalLongitude', 'year', 'month', 'day', 'recordedBy', 'id', 'references', 'individualCount', 'locality']]
clienttemplate['Source*'] = np.where(clienttemplate['Latitude*'].isna()|clienttemplate['Longitude*'].isna(), '', 'reported')
clienttemplate['Accuracy*'] = np.where(clienttemplate['Source*'] == 'reported', 'accurate', '') #
clienttemplate['Reference 2']= atlas_url #, 'month', 'day', 'recordedBy', 'id', 'references', 'individualCount', 'locality'
clienttemplate['Record Type*'] = np.where(masterdataframe['collectionCode'].isna(), '', 'specimen')
clienttemplate['Year Accuracy of specimen*'] = masterdataframe['year']
